In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
id_information = pd.read_csv('id_information_mmsr.tsv', sep='\t')
id_genres = pd.read_csv('id_genres_mmsr.tsv', sep='\t')
id_bert = pd.read_csv('id_bert_mmsr.tsv', sep='\t')
id_lyrics_tfidf = pd.read_csv('id_lyrics_tf-idf_mmsr.tsv', sep='\t')
id_lyrics_word2vec = pd.read_csv('id_lyrics_word2vec_mmsr.tsv', sep='\t')

In [5]:
id_lyrics_tfidf['id']

0        9jbSytob9XRzwvB6
1        Njp6JPM8vitbhVJU
2        h48f46ZsT9h0Z5Dm
3        ZmXVK43zlqdeq6z8
4        PV5EXN6AIVBqvsLO
               ...       
76110    R4VMVAxVOAEWBjgg
76111    YgII1tHAaAnh14Kf
76112    76qenAf8gYlH4pfq
76113    9F8jQjeibAuZinEP
76114    njEze6oZPy3MX2dN
Name: id, Length: 76115, dtype: object

In [6]:
artist = 'Dua Lipa'
song_query = 'New Rules (Initial Talk Remix)'
song = id_information.loc[(id_information['artist']==artist)&(id_information['song']==song_query)]

In [7]:
query_id = song.iloc[0]['id']

In [8]:
tfidf_query = id_lyrics_tfidf.loc[id_lyrics_tfidf['id']==song.iloc[0]['id']]
tfidf_query = np.array(tfidf_query.drop(columns=['id']))

In [9]:
tfidf_subset = id_lyrics_tfidf#.head(10000)
tfidf_subset

,id,abl,accept,across,act,addict,afraid,age,ago,ah,...,yea,yeah,year,yellow,yes,yesterday,yet,yo,young,youth
0,9jbSytob9XRzwvB6,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.150511
1,Njp6JPM8vitbhVJU,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,h48f46ZsT9h0Z5Dm,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.149783,0.0,0.0,0.0,0.000000
3,ZmXVK43zlqdeq6z8,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,PV5EXN6AIVBqvsLO,0.0,0.0,0.000000,0.0,0.000000,0.327025,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76110,R4VMVAxVOAEWBjgg,0.0,0.0,0.043412,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
76111,YgII1tHAaAnh14Kf,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
76112,76qenAf8gYlH4pfq,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
76113,9F8jQjeibAuZinEP,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [10]:
#def cosine_similarity(x,y):
#    return np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [11]:
subset_ids = tfidf_subset[['id']].copy()
subset_ids

,id
0,9jbSytob9XRzwvB6
1,Njp6JPM8vitbhVJU
2,h48f46ZsT9h0Z5Dm
3,ZmXVK43zlqdeq6z8
4,PV5EXN6AIVBqvsLO
...,...
76110,R4VMVAxVOAEWBjgg
76111,YgII1tHAaAnh14Kf
76112,76qenAf8gYlH4pfq
76113,9F8jQjeibAuZinEP


In [12]:
tfidf_no_id = np.array(tfidf_subset.drop(columns=['id']))
cosine = cosine_similarity(tfidf_query, tfidf_no_id)
subset_ids.insert(1,'cosine', cosine[0])
subset_ids

,id,cosine
0,9jbSytob9XRzwvB6,0.057755
1,Njp6JPM8vitbhVJU,0.019939
2,h48f46ZsT9h0Z5Dm,0.062809
3,ZmXVK43zlqdeq6z8,0.110308
4,PV5EXN6AIVBqvsLO,0.042527
...,...,...
76110,R4VMVAxVOAEWBjgg,0.027622
76111,YgII1tHAaAnh14Kf,0.000000
76112,76qenAf8gYlH4pfq,0.098476
76113,9F8jQjeibAuZinEP,0.037870


In [13]:
subset_ids= subset_ids.sort_values('cosine', ascending=False)

In [14]:
sorted_cosine = subset_ids.iloc[0: , :]
sorted_cosine

,id,cosine
69128,zzznMjZAKnJJXQSj,1.000000
45382,m3bU7wEiG8i3QgLU,1.000000
52360,gSDEde5fekD0MWlk,0.966519
60655,VxUQdLc5okNKJStp,0.459405
42169,YeNBs5M5SgaiJwMO,0.446333
...,...,...
54443,ZSy6SvoHjiRG0nwO,0.000000
60382,CmwEAIc6vATWKEqN,0.000000
52244,SR5GYmfvhKlhbNJV,0.000000
68836,A6iqnafwY8Yy5pWI,0.000000


In [15]:
top_10 = sorted_cosine.head(10)
top_10

,id,cosine
69128,zzznMjZAKnJJXQSj,1.000000
45382,m3bU7wEiG8i3QgLU,1.000000
52360,gSDEde5fekD0MWlk,0.966519
60655,VxUQdLc5okNKJStp,0.459405
42169,YeNBs5M5SgaiJwMO,0.446333
60359,fKMI8NwzTaL2psTG,0.420308
6443,E7uZQXT21LTv1pbw,0.411529
43899,1raObPYSGFUis50J,0.404821
16972,aTzS0RJbn5d8Hfsm,0.396373
21137,1Y6EGNZVrpANBgpf,0.392309


In [16]:
top_10_ids = np.array(top_10['id'])

In [17]:
for idx in top_10_ids:
    print(id_information.loc[(id_information['id'] == idx)][['artist', 'song']])

         artist                            song
76113  Dua Lipa  New Rules (Initial Talk Remix)
         artist                            song
58942  Dua Lipa  New Rules - Initial Talk Remix
         artist       song
52034  Dua Lipa  New Rules
              artist                           song
39323  Lenny Kravitz  Let Love Rule (Justice Remix)
        artist song
42582  J. Cole  ATM
                   artist         song
50654  Jefferson Starship  Count on Me
                artist         song
17479  Whitney Houston  Count On Me
          artist         song
2424  Bruno Mars  Count on Me
        artist      song
44737  Ramones  Commando
     artist    song
2019   Kllo  Virtue


In [18]:
top_10_meta = id_information.loc[id_information['id'].isin(top_10_ids)]  # not ordered

In [19]:
query_genre = id_genres.loc[id_genres['id']==query_id]['genre']
query_genre

76113    ['synthpop', 'pop']
Name: genre, dtype: object

In [20]:
def get_genre(song_id):
    genres = id_genres.loc[id_genres['id']==song_id]['genre'].tolist()[0]
    return genres

In [21]:
def string_to_list(string):
    lst = string[1:-1].split(', ')
    return lst

In [22]:
query_genre = get_genre(query_id)
print(query_genre)
query_genre = string_to_list(query_genre)

['synthpop', 'pop']


In [23]:

tp = 0
for idx in top_10_ids:
    genres = get_genre(idx)
    genre_lst = string_to_list(genres)[0]
    if any(item in query_genre[0] for item in genre_lst):
        tp += 1

precision = tp/len(top_10_ids)
print(precision)

1.0


In [3]:
def get_genre(song_id):
    genres = id_genres.loc[id_genres['id']==song_id]['genre'].tolist()[0]
    return genres

def string_to_list(string):
    lst = string[1:-1].split(', ')
    return lst

def get_top_k(artist, song_query, k:int):
    song = id_information.loc[(id_information['artist']==artist)&(id_information['song']==song_query)]
    query_id = song.iloc[0]['id']
    tfidf_query = id_lyrics_tfidf.loc[id_lyrics_tfidf['id']==song.iloc[0]['id']]
    tfidf_query = np.array(tfidf_query.drop(columns=['id']))
    tfidf_ids = id_lyrics_tfidf[['id']].copy()
    tfidf_no_id = np.array(id_lyrics_tfidf.copy().drop(columns=['id']))
    cosine = cosine_similarity(tfidf_query, tfidf_no_id)
    tfidf_ids.insert(1,'cosine', cosine[0])
    tfidf_ids= tfidf_ids.sort_values('cosine', ascending=False)
    sorted_cosine = tfidf_ids.iloc[1: , :]
    top_k = sorted_cosine.head(k)
    top_k_ids = np.array(top_k['id'])
    query_genre = get_genre(query_id)
    query_genre = string_to_list(query_genre)
    
    tp = 0
    for idx in top_k_ids:
        genres = get_genre(idx)
        genre_lst = string_to_list(genres)
        if any(item in query_genre for item in genre_lst):
            tp += 1
    precision = tp/len(top_k_ids)  # somethings wrong
    
    for idx in top_k_ids:
        print(id_information.loc[(id_information['id'] == idx)][['artist', 'song']])
    
    return precision

In [4]:
print(get_top_k('Etta James', 'I Just Want to Make Love to You', 10))

             artist                             song
44262  Muddy Waters  I Just Want To Make Love To You
                 artist             song
61869  Rachael Yamagata  Be Be Your Love
        artist               song
73660  NCT 127  Replay (PM 01:27)
                 artist              song
34292  Transvision Vamp  I Want Your Love
       artist                             song
52291  Foghat  I Just Want to Make Love to You
            artist           song
70542  Jakob Ogawa  All Your Love
       artist                                               song
59396  Foghat  I Just Want to Make Love To You - 2016 Remastered
                   artist                             song
17750  The Rolling Stones  I Just Want to Make Love to You
        artist             song
36620  Madonna  Justify My Love
                 artist         song
70349  Carly Rae Jepsen  Gimmie Love
0.7
